In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [2]:
data = """Twinkle, twinkle, little star,
          How I wonder what you are!
          Up above the world so high,
          Like a diamond in the sky.

          When the blazing sun is gone,
          When he nothing shines upon,
          Then you show your little light,
          Twinkle, twinkle, all the night."""

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
vocab_size = len(tokenizer.word_index) + 1

In [4]:
sequences = []
for line in data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        sequences.append(n_gram_sequence)
     

In [5]:
max_sequence_len = max([len(seq) for seq in sequences])
sequences = np.array(pad_sequences(sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
X = sequences[:, :-1]
y = sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)

In [7]:
# Define the RNN model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=max_sequence_len-1))
model.add(LSTM(100))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100
2/2 [==============================] - 6s 17ms/step - loss: 3.5829 - accuracy: 0.0278
Epoch 2/100
2/2 [==============================] - 0s 29ms/step - loss: 3.5796 - accuracy: 0.1389
Epoch 3/100
2/2 [==============================] - 0s 32ms/step - loss: 3.5772 - accuracy: 0.1111
Epoch 4/100
2/2 [==============================] - 0s 47ms/step - loss: 3.5748 - accuracy: 0.1111
Epoch 5/100
2/2 [==============================] - 0s 28ms/step - loss: 3.5722 - accuracy: 0.1111
Epoch 6/100
2/2 [==============================] - 0s 22ms/step - loss: 3.5693 - accuracy: 0.1111
Epoch 7/100
2/2 [==============================] - 0s 21ms/step - loss: 3.5664 - accuracy: 0.1111
Epoch 8/100
2/2 [==============================] - 0s 10ms/step - loss: 3.5630 - accuracy: 0.1111
Epoch 9/100
2/2 [==============================] - 0s 29ms/step - loss: 3.5592 - accuracy: 0.1111
Epoch 10/100
2/2 [==============================] - 0s 89ms/step - loss: 3.5555 - accuracy: 0.1111
Epoch 11/100
2/2 [=

In [9]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)[0]
        predicted_index = np.argmax(predicted_probs)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

generated_text = generate_text("The", 10, model, max_sequence_len)
print(generated_text)


The twinkle the the sky gone high gone high sky sky
